# Pulse Width Modulation

Daniel Rothfusz

Goal: Emulate analog voltage ranges on a digital output to get varying brightness on an RGB LED

In [ ]:
from pynq.overlays.base import BaseOverlay
import time
from datetime import datetime
import asyncio
base = BaseOverlay("base.bit")

In [ ]:
%%microblaze base.PMODB

#include "gpio.h"
#include "pyprintf.h"

//Function to turn on/off a selected pin of PMODB
unsigned int write_gpio(unsigned int pin, unsigned int val){
    if (pin > 3 || pin < 1)
    {
        pyprintf("ERROR: invalid pin");
        return 0;
    }
    if (val > 1)
    {
        pyprintf("ERROR: invalid value");
    }
    gpio pin_out = gpio_open(pin);
    gpio_set_direction(pin_out, GPIO_OUT);
    gpio_write(pin_out, val);
    return 0;
}

//Function to read the value of a selected pin of PMODB
unsigned int read_gpio(unsigned int pin){
    gpio pin_in = gpio_open(pin);
    gpio_set_direction(pin_in, GPIO_IN);
    return gpio_read(pin_in);
}

//Function to reset all GPIO on PMODB
unsigned int reset_all_gpio()
{
    for (unsigned int i=0; i <= 7; i++)
    {
        gpio temp_gpio = gpio_open(i);
        gpio_close(temp_gpio);
    }
    return 0;
}

### Part 1: Reset GPIO


In [ ]:
reset_all_gpio()

### Part 2-3: PWM GPIO code
This section sets up async functions to run a single pin on the GPIO in PWM operation. It assumes the user has plugged their RGB LED into pins 1-3 on PMODB with the ground pin already plugged into a ground port.
I found I could not perceive the flashing shortly after ~50 Hz

In [ ]:
cond = True

btns = base.btns_gpio

async def enable_pwm(pin, duty_cycle, frequency):
    global cond
    if (pin < 0 or pin > 2):
        cond=False
        raise Exception(f"Invalid pin selected: {pin}")
    if (duty_cycle < 0 or duty_cycle > 250):
        cond=False
        raise Exception(f"Invalid duty cycle: {duty_cycle}")
    if frequency > 70:
        cond=False
        raise Exception(f"Invalid frequency: {frequency}")
    # Set all output pins to output pins set to low
    print(f"PWM enabled at {duty_cycle}% duty_cycle and {frequency} Hz")
    write_gpio(1,0)
    write_gpio(2,0)
    write_gpio(3,0)
    period=1.0/frequency
    while cond:
        if duty_cycle != 0:
            write_gpio(pin,1)
            await asyncio.sleep((duty_cycle/100.0)*period)
        if duty_cycle != 100:
            write_gpio(pin,0)
            await asyncio.sleep((1-(duty_cycle/100.0))*period)

async def get_btns(_loop):
    global cond
    while cond:
        await asyncio.sleep(0.01)
        if btns[0].read() != 0:
            cond = False     
    await asyncio.sleep(0.1)
    _loop.stop()


### Part 4: PWM brightness

In [ ]:
### 0% brightness
cond = True
loop = asyncio.new_event_loop()
loop.create_task(get_btns(loop))
loop.create_task(enable_pwm(2, 0, 70.0))
try:
    loop.run_forever()
finally:
    loop.run_until_complete(loop.shutdown_asyncgens())
loop.close()
reset_all_gpio()

In [ ]:
### 25% brightness
cond = True
loop = asyncio.new_event_loop()
loop.create_task(get_btns(loop))
loop.create_task(enable_pwm(2, 10, 70.0))
try:
    loop.run_forever()
finally:
    loop.run_until_complete(loop.shutdown_asyncgens())
loop.close()
reset_all_gpio()

In [ ]:
### 50% brightness
cond = True
loop = asyncio.new_event_loop()
loop.create_task(get_btns(loop))
loop.create_task(enable_pwm(2, 30, 70.0))
try:
    loop.run_forever()
finally:
    loop.run_until_complete(loop.shutdown_asyncgens())
loop.close()
reset_all_gpio()

In [ ]:
### 75% brightness
cond = True
loop = asyncio.new_event_loop()
loop.create_task(get_btns(loop))
loop.create_task(enable_pwm(2, 55, 70.0))
try:
    loop.run_forever()
finally:
    loop.run_until_complete(loop.shutdown_asyncgens())
loop.close()
reset_all_gpio()

In [ ]:
### 100% brightness
cond = True
loop = asyncio.new_event_loop()
loop.create_task(get_btns(loop))
loop.create_task(enable_pwm(2, 100, 70.0))
try:
    loop.run_forever()
finally:
    loop.run_until_complete(loop.shutdown_asyncgens())
loop.close()
reset_all_gpio()

In [ ]:
### Set OFF as a reference (to set all pins to output mode rather than tristate)
write_gpio(3,0)
write_gpio(2,0)
write_gpio(1,0)

In [ ]:
### Set ON as a reference
write_gpio(3,0)
write_gpio(2,1)
write_gpio(1,0)

### Part 5 - Duty Cycle % vs Brightness Plot
This section I included a slider to enable more granular measurements as I tried to develop a plot of brightness over the duty cycle range

In [ ]:
import ipywidgets as widgets

In [ ]:
sliderval = 50

slider = widgets.IntSlider(
    value=sliderval,
    min=0,
    max=100,
    step=1,
    description='Duty Cycle %:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

slider

In [ ]:
cond = True
loop = asyncio.new_event_loop()
loop.create_task(get_btns(loop))
loop.create_task(enable_pwm(1, slider.value, 70.0))
try:
    loop.run_forever()
finally:
    loop.run_until_complete(loop.shutdown_asyncgens())
loop.close()
reset_all_gpio()

### Part 6 - Color Changing Blinking PWM
In this section, I change the color via buttons while keeping a fixed PWM duty cycle and frequency and toggling on/off every 1 second

In [ ]:
cond = True

btns = base.btns_gpio

# Blue=1, Green=2, Red=3
active_pin = 1

blink_on = True

async def enable_blinking_pwm(duty_cycle, frequency):
    global cond, blink_on
    if (duty_cycle < 0 or duty_cycle > 100):
        cond=False
        raise Exception(f"Invalid duty cycle: {duty_cycle}")
    if frequency > 100:
        cond=False
        raise Exception(f"Invalid frequency: {frequency}")
    # Set all output pins to output pins set to low
    print(f"PWM enabled at {duty_cycle}% duty_cycle and {frequency} Hz")
    period=1.0/frequency
    while cond:
        # Enable PWM for selected pin if during a blink and PWM is a nonzero duty cycle
        if duty_cycle != 0 and blink_on:
            write_gpio(active_pin,1)
            await asyncio.sleep((duty_cycle/100.0)*period)
        if duty_cycle != 100 or not blink_on:
            write_gpio(1,0)
            write_gpio(2,0)
            write_gpio(3,0)
            await asyncio.sleep((1-(duty_cycle/100.0))*period)
            
async def blink_led():
    global cond, blink_on
    while cond:
        blink_on = True
        await asyncio.sleep(1)
        blink_on = False
        await asyncio.sleep(1)

async def get_blinking_btns(_loop):
    global cond, active_pin
    while cond:
        await asyncio.sleep(0.01)
        if btns[0].read() != 0:
            active_pin = 3 # Red
        elif btns[1].read () != 0:
            active_pin = 2 # Green
        elif btns[2].read () != 0:
            active_pin = 1 # Blue
        elif btns[3].read () != 0:
            cond = False
            blink_on = False
    await asyncio.sleep(2)
    _loop.stop()

In [ ]:
cond=True

loop = asyncio.new_event_loop()
loop.create_task(get_blinking_btns(loop))
loop.create_task(enable_blinking_pwm(30, 70.0))
loop.create_task(blink_led())
try:
    loop.run_forever()
finally:
    loop.run_until_complete(loop.shutdown_asyncgens())
loop.close()